In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [7]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_10월.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [8]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [9]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

100%|██████████| 24376/24376 [54:45<00:00,  7.42it/s]


,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/12__02/222916170832,"캠핑일기, 가평 고릴라펜션 캠핑장",2022.10.31.,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니\n깜빡해버려서 몰아서 쓰는 중이다....,0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다 이...,[(10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다...
1,https://blog.naver.com/skawn1012/222916169496,애도. 그리고 할로윈,2022.10.31.,주일 아침 기상 알람이 울리기 전 아빠에게로부터 전화가 왔다.\n어젯밤 들어오지 않...,1,주일 아침 기상 알람이 울리기 전 아빠에게로부터 전화가 왔다 어젯밤 들어오지 않은 ...,[(주일 아침 기상 알람이 울리기 전 아빠에게로부터 전화가 왔다 어젯밤 들어오지 않...
2,https://blog.naver.com/sngh_log/222916168962,"미국 서부여행 | 샌프란시스코, 나파 & 소노마 밸리 와이너리...",2022.10.31.,"San Francisco, CA\nUnited States of America\n안...",2,San Francisco CA United States of America 안녕하세...,[(San Francisco CA United States of America 안녕...
3,https://blog.naver.com/plus3861/222916167331,"2022년 10월의 마지막 밤에, 10월을 보내며, 2022년 10개월의...",2022.10.31.,세월은 유수와 같다고 하더니\n2022년을 시작한 게 엊그제 같은데 벌써 10달이 ...,3,세월은 유수와 같다고 하더니 2022년을 시작한 게 엊그제 같은데 벌써 10달이 지...,[(세월은 유수와 같다고 하더니 2022년을 시작한 게 엊그제 같은데 벌써 10달이...
4,https://blog.naver.com/juun306/222916166316,7. 리움미술관 : 구름산책자,2022.10.31.,저번에 리움을 방문했을 때에 시간이 모자라 기획전을 관람하지 못했다.\n리움에서 이...,4,저번에 리움을 방문했을 때에 시간이 모자라 기획전을 관람하지 못했다 리움에서 이뤄지...,"[(저번에 리움을 방문했을 때에 시간이 모자라 기획전을 관람하지 못했다, 0, 37..."
...,...,...,...,...,...,...,...
24371,https://blog.naver.com/jyeight/222888768603,"[밴쿠버, DAY 2] 진짜 진짜 밴쿠버에서의 첫 날 | SFU...",2022.10.01.,"DAY 2\n2022.07.03.\n시차로 인해, 둘쨋 날이지만 3일입니다요\n장거...",24371,DAY 2 2022 07 03 시차로 인해 둘쨋 날이지만 3일입니다요 장거리 비행으...,"[(DAY 2 2022 07 03 시차로 인해 둘쨋 날이지만 3일입니다요, 0, 3..."
24372,https://blog.naver.com/cjyse2012/222888767536,"[이집트여행] 카이로 3일차, 모마 시티투어(쓰레기 마을, 동굴...",2022.10.01.,"카이로 3일차, 모마 시티투어 빡세다. 아침부터 오후 4~5시까지 일정이 빽빽하다....",24372,카이로 3일차 모마 시티투어 빡세다 아침부터 오후 4 5시까지 일정이 빽빽하다 카이...,"[(카이로 3일차 모마 시티투어 빡세다 아침부터 오후 4 5시까지 일정이 빽빽하다,..."
24373,https://blog.naver.com/chakeun/222888765895,인류 최초의 동력비행이 성공한 장소인 라이트형제...,2022.10.01.,가끔 줄여서 '노바(NoVA)'라고 부르는 워싱턴DC 인근의 북부 버지니아(Nort...,24373,가끔 줄여서 노바 NoVA 라고 부르는 워싱턴DC 인근의 북부 버지니아 Northe...,[(가끔 줄여서 노바 NoVA 라고 부르는 워싱턴DC 인근의 북부 버지니아 Nort...
24374,https://blog.naver.com/indigotablet/222888745028,[22/09/26-30] 금목서 헌터,2022.10.01.,블로그\nbusan diary 2022 37개의 글\nbusan diary 2022...,24374,블로그 busan diary 2022 37개의 글 busan diary 2022 목...,[(블로그 busan diary 2022 37개의 글 busan diary 2022...


In [10]:
import pickle
# 데이터프레임을 피클 파일로 저장함으로써 데이터타입 보존
with open('10월_문장분리.pkl', 'wb') as file:
    pickle.dump(df, file)

In [2]:
import pickle
# 저장된 피클 파일을 불러오기 익일 재실행 경우, 이 코드만 돌리면 시간절약 가능.
with open('10월_문장분리.pkl', 'rb') as file:
    df = pickle.load(file)
df

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/12__02/222916170832,"캠핑일기, 가평 고릴라펜션 캠핑장",2022.10.31.,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니\n깜빡해버려서 몰아서 쓰는 중이다....,0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다 이...,[(10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다...
1,https://blog.naver.com/skawn1012/222916169496,애도. 그리고 할로윈,2022.10.31.,주일 아침 기상 알람이 울리기 전 아빠에게로부터 전화가 왔다.\n어젯밤 들어오지 않...,1,주일 아침 기상 알람이 울리기 전 아빠에게로부터 전화가 왔다 어젯밤 들어오지 않은 ...,[(주일 아침 기상 알람이 울리기 전 아빠에게로부터 전화가 왔다 어젯밤 들어오지 않...
2,https://blog.naver.com/sngh_log/222916168962,"미국 서부여행 | 샌프란시스코, 나파 & 소노마 밸리 와이너리...",2022.10.31.,"San Francisco, CA\nUnited States of America\n안...",2,San Francisco CA United States of America 안녕하세...,[(San Francisco CA United States of America 안녕...
3,https://blog.naver.com/plus3861/222916167331,"2022년 10월의 마지막 밤에, 10월을 보내며, 2022년 10개월의...",2022.10.31.,세월은 유수와 같다고 하더니\n2022년을 시작한 게 엊그제 같은데 벌써 10달이 ...,3,세월은 유수와 같다고 하더니 2022년을 시작한 게 엊그제 같은데 벌써 10달이 지...,[(세월은 유수와 같다고 하더니 2022년을 시작한 게 엊그제 같은데 벌써 10달이...
4,https://blog.naver.com/juun306/222916166316,7. 리움미술관 : 구름산책자,2022.10.31.,저번에 리움을 방문했을 때에 시간이 모자라 기획전을 관람하지 못했다.\n리움에서 이...,4,저번에 리움을 방문했을 때에 시간이 모자라 기획전을 관람하지 못했다 리움에서 이뤄지...,"[(저번에 리움을 방문했을 때에 시간이 모자라 기획전을 관람하지 못했다, 0, 37..."
...,...,...,...,...,...,...,...
24371,https://blog.naver.com/jyeight/222888768603,"[밴쿠버, DAY 2] 진짜 진짜 밴쿠버에서의 첫 날 | SFU...",2022.10.01.,"DAY 2\n2022.07.03.\n시차로 인해, 둘쨋 날이지만 3일입니다요\n장거...",24371,DAY 2 2022 07 03 시차로 인해 둘쨋 날이지만 3일입니다요 장거리 비행으...,"[(DAY 2 2022 07 03 시차로 인해 둘쨋 날이지만 3일입니다요, 0, 3..."
24372,https://blog.naver.com/cjyse2012/222888767536,"[이집트여행] 카이로 3일차, 모마 시티투어(쓰레기 마을, 동굴...",2022.10.01.,"카이로 3일차, 모마 시티투어 빡세다. 아침부터 오후 4~5시까지 일정이 빽빽하다....",24372,카이로 3일차 모마 시티투어 빡세다 아침부터 오후 4 5시까지 일정이 빽빽하다 카이...,"[(카이로 3일차 모마 시티투어 빡세다 아침부터 오후 4 5시까지 일정이 빽빽하다,..."
24373,https://blog.naver.com/chakeun/222888765895,인류 최초의 동력비행이 성공한 장소인 라이트형제...,2022.10.01.,가끔 줄여서 '노바(NoVA)'라고 부르는 워싱턴DC 인근의 북부 버지니아(Nort...,24373,가끔 줄여서 노바 NoVA 라고 부르는 워싱턴DC 인근의 북부 버지니아 Northe...,[(가끔 줄여서 노바 NoVA 라고 부르는 워싱턴DC 인근의 북부 버지니아 Nort...
24374,https://blog.naver.com/indigotablet/222888745028,[22/09/26-30] 금목서 헌터,2022.10.01.,블로그\nbusan diary 2022 37개의 글\nbusan diary 2022...,24374,블로그 busan diary 2022 37개의 글 busan diary 2022 목...,[(블로그 busan diary 2022 37개의 글 busan diary 2022...


In [3]:
# 데이터프레임을 10개 단위로 나누기
chunk_size = 50
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [14]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_10월_전처리.csv", index=False, encoding='UTF-8')

In [4]:
import pandas as pd
df2 = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_10월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다,0
1,이날 날씨가 너무 좋다 보니 캠핑장과 펜션은 당연히 꽉 차버렸고 캠핑장 사장님 찬스...,0
2,친한 언니들 A언니와 B언니 부부와 함께 갔던 날 우리 부부까지 합치면 다섯 명 이...,0
3,푸릇푸릇 한 밤나무들이 인상적이다 주차장에도 차가 정 말 많았다,0
4,차가 많이 막혀서 일찍 출발했는데도 시간이 꽤 걸렸다,0
...,...,...
1050359,요즘은 낯설지 않은 모습이죠,23064
1050360,백색소음 효과 예전 우리네는 책을 들고 방 밖으로 나올라치면 어김없는 등짝 스매싱 ...,23064
1050361,손보지 않은 채로 툭 던져진 일상 분위기 탓일까 감성 자극 물씬 주인장 마음 방문하...,23064
1050362,오늘은 우리 차지 마지막 빵 모두 담아와 맛있게 순삭 오늘 같은 내일이 쭉 이어지길...,23064


In [5]:
df2 = df2[df2['org_idx'] < 23050]
df2

,split_str,org_idx
0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다,0
1,이날 날씨가 너무 좋다 보니 캠핑장과 펜션은 당연히 꽉 차버렸고 캠핑장 사장님 찬스...,0
2,친한 언니들 A언니와 B언니 부부와 함께 갔던 날 우리 부부까지 합치면 다섯 명 이...,0
3,푸릇푸릇 한 밤나무들이 인상적이다 주차장에도 차가 정 말 많았다,0
4,차가 많이 막혀서 일찍 출발했는데도 시간이 꽤 걸렸다,0
...,...,...
1049772,조선황실 계보도 태조어진 봉안 행렬 전동성당이 잘 보이네요,23049
1049773,요즘은 남성이 여성 한복을 여성이 남성한복을 입기도 하네요,23049
1049774,전주한옥마을해설투어 경기전 해설투어 전라북도 태조 이성계 역사탐방 전주한옥마을 경기...,23049
1049775,전라북도 청념축제가 경기전 입구 왼쪽에서 진행되고 있어요,23049


In [6]:
df2['org_idx'].tolist()[-1]//50+2

462

In [7]:
for a in range(df2['org_idx'].tolist()[-1]//50+2,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_10월_전처리.csv", index=False, encoding='UTF-8')
df2

100%|██████████| 26/26 [02:50<00:00,  6.55s/it]


,split_str,org_idx
0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다,0
1,이날 날씨가 너무 좋다 보니 캠핑장과 펜션은 당연히 꽉 차버렸고 캠핑장 사장님 찬스...,0
2,친한 언니들 A언니와 B언니 부부와 함께 갔던 날 우리 부부까지 합치면 다섯 명 이...,0
3,푸릇푸릇 한 밤나무들이 인상적이다 주차장에도 차가 정 말 많았다,0
4,차가 많이 막혀서 일찍 출발했는데도 시간이 꽤 걸렸다,0
...,...,...
1113264,관람을 마치고 나오면 야외에도 이렇게 좌불상들이 있다,24375
1113265,진짜 중국 스케일 최고 불교는 아니지만 중국 도시를 여행 할 때마다 사찰과 같은 불...,24375
1113266,뭔가 현실세계가 아닌 느낌이랄까,24375
1113267,그래서 세계인이 주목하는 미래형 불교사찰인거겠지 이정도는 되어야 중국 3대 불교 성...,24375


In [8]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_10월_전처리.csv", index=False, encoding='UTF-8')